# Regression

In this notebook, I'll demonstrate workflows for performing linear regression. For this purpose, I'll be using the 'Tips' dataset included with Seaborn

In [2]:
# Load libraries

# General DS Libraries
import seaborn as sns
import pandas as pd
import numpy as np

# Models
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor

# Pre-processing and Scoring
from sklearn.model_selection import cross_val_score, RandomizedSearchCV

Here's the load for the "Tips" dataset and a quick look at it

In [4]:
tips = sns.load_dataset('tips')
print(tips.info())
print(tips.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244 entries, 0 to 243
Data columns (total 7 columns):
total_bill    244 non-null float64
tip           244 non-null float64
sex           244 non-null category
smoker        244 non-null category
day           244 non-null category
time          244 non-null category
size          244 non-null int64
dtypes: category(4), float64(2), int64(1)
memory usage: 7.2 KB
None
   total_bill   tip     sex smoker  day    time  size
0       16.99  1.01  Female     No  Sun  Dinner     2
1       10.34  1.66    Male     No  Sun  Dinner     3
2       21.01  3.50    Male     No  Sun  Dinner     3
3       23.68  3.31    Male     No  Sun  Dinner     2
4       24.59  3.61  Female     No  Sun  Dinner     4
